In [1]:
#update the metadata info
import scanpy as sc #for scanpy >= 1.3.7
import anndata as ann
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
import logging
import os
import glob
import matplotlib
import math

In [2]:
adata_savepath = '/domino/edv/id-td-virology/Zhiyuan/public/Gut_remap/processed_data/annotated.h5ad'
adata= sc.read_h5ad(adata_savepath)

In [3]:
savepath = '/domino/edv/id-td-virology/Zhiyuan/public/Gut_remap/processed_data/CD4.h5ad'
adata_CD4 = sc.read_h5ad(savepath)
savepath = '/domino/edv/id-td-virology/Zhiyuan/public/Gut_remap/processed_data/CD8.h5ad'
adata_CD8 = sc.read_h5ad(savepath)

In [4]:

df = adata.obs.copy()
df['subtype'] = df['FinalAnnotation'].astype('object')
df.loc[adata_CD4.obs.index, 'subtype'] = adata_CD4.obs.loc[:,'subtype'].astype('object')
df.loc[adata_CD8.obs.index, 'subtype'] = adata_CD8.obs.loc[:,'subtype'].astype('object')

In [5]:
df.groupby('subtype').size()

subtype
Activated B               2472
CD4+ Naive/cm            26404
CD4+ Th1_like             9353
CD4+ Treg CTLA4++         2597
CD4+ Treg SELL++          2289
CD4+ intermediate        23875
CD8+ MAIT                17091
CD8+ Naive                9585
CD8+ Tcm                  2196
CD8+ Tem                 38377
CD8+ Temra/Teff          24518
CD8+ Tex                  5045
DN-Tcell                   529
Doublets                 15206
KupfferCell                178
LSEC                       653
Macrophage                 992
Memory B                  9673
MonocytesClassical       20409
MonocytesNonClassical     3400
NK                       33361
NK-Resident              12688
Naive B                  12362
Plasma                    1627
Platelets                 1833
Proliferating             2365
cDC1                      1212
cDC2                      1544
gdT                       6283
pDC                        647
unknown                   4796
dtype: int64

In [9]:
#change subtype names
df.loc[df.subtype == 'CD4+ Naive/cm', 'subtype'] = 'CD4+ T Naïve/Central Memory'
df.loc[df.subtype == 'CD4+ Th1_like', 'subtype'] = 'CD4+ T Effector'
df.loc[df.subtype == 'CD4+ Treg CTLA4++', 'subtype'] = 'CD4+ Resident Treg'
df.loc[df.subtype == 'CD4+ Treg SELL++', 'subtype'] = 'CD4+ Circulating Treg'
df.loc[df.subtype == 'CD4+ intermediate', 'subtype'] = 'CD4+ T Effector Memory'
df.loc[df.subtype == 'CD8+ Naive', 'subtype'] = 'CD8+ T Naïve'
df.loc[df.subtype == 'CD8+ Tcm', 'subtype'] = 'CD8+ T Central Memory'
df.loc[df.subtype == 'CD8+ Tem', 'subtype'] = 'CD8+ T Effector Memory'
df.loc[df.subtype == 'CD8+ Tem', 'subtype'] = 'CD8+ T Effector Memory'
df.loc[df.subtype == 'CD8+ Temra/Teff', 'subtype'] = 'CD8+ T Effctor'
df.loc[df.subtype == 'CD8+ Tex', 'subtype'] = 'CD8+ T Exhausted'

df.loc[df.subtype == 'Naive B', 'subtype'] = 'B Naïve'
df.loc[df.subtype == 'Memory B', 'subtype'] = 'B Memory'
df.loc[df.subtype == 'Activated B', 'subtype'] = 'B Activated'




In [17]:
df.groupby(['SecondAnnotation', 'subtype'], observed = True).size()

SecondAnnotation  subtype                    
CD8+              CD8+ MAIT                      16929
                  CD8+ T Central Memory            341
                  CD8+ T Effctor                  8979
                  CD8+ T Effector Memory         34929
                  CD8+ T Exhausted                5043
                  CD8+ T Naïve                    9089
NK                NK                             33361
                  NK-Resident                    12688
Doublets          Doublets                       15206
CD3+              DN-Tcell                         529
                  gdT                             6283
CD4+              CD4+ Circulating Treg           2289
                  CD4+ Resident Treg              2597
                  CD4+ T Effector                 9353
                  CD4+ T Effector Memory         23875
                  CD4+ T Naive/Central memory    26404
                  CD8+ MAIT                        162
                  C

In [18]:
test = df.loc[(df.SecondAnnotation == 'CD4+')].copy()

ind = [x for x in test.index if test.loc[x,'subtype'].startswith('CD8+')]

In [21]:
df.loc[ind, 'SecondAnnotation'] = 'CD8+'

In [22]:
df.groupby(['SecondAnnotation', 'subtype'], observed = True).size()

SecondAnnotation  subtype                    
CD8+              CD8+ MAIT                      17091
                  CD8+ T Central Memory           2196
                  CD8+ T Effctor                 24518
                  CD8+ T Effector Memory         38377
                  CD8+ T Exhausted                5045
                  CD8+ T Naïve                    9585
NK                NK                             33361
                  NK-Resident                    12688
Doublets          Doublets                       15206
CD3+              DN-Tcell                         529
                  gdT                             6283
CD4+              CD4+ Circulating Treg           2289
                  CD4+ Resident Treg              2597
                  CD4+ T Effector                 9353
                  CD4+ T Effector Memory         23875
                  CD4+ T Naive/Central memory    26404
Proliferating     Proliferating                   2365
Macrophage        K

In [23]:
adata.obs.loc[df.index, 'subtype_final'] = df.loc[:,'subtype']
adata.obs.loc[df.index, 'SecondAnnotation'] = df.loc[:,'SecondAnnotation']

In [26]:
#save the updated h5ad adata
path = '/domino/edv/id-td-virology/Zhiyuan/public/Gut_remap/processed_data/annotated_final.h5ad'
adata.write(path)

In [32]:
#save metadata for Nadia
df_2 = adata.obs[['cell_quality','SecondAnnotation', 'subtype_final']].copy()

In [33]:
df_2.groupby(['cell_quality','SecondAnnotation', 'subtype_final'], observed = True).size()

cell_quality  SecondAnnotation  subtype_final              
high          CD8+              CD8+ T Effector Memory         38377
                                CD8+ MAIT                      17091
                                CD8+ T Effctor                 24518
                                CD8+ T Exhausted                5045
                                CD8+ T Central Memory           2196
                                CD8+ T Naïve                    9585
              NK                NK                             33361
                                NK-Resident                    12688
              Doublets          Doublets                       15206
              CD3+              gdT                             6283
                                DN-Tcell                         529
              CD4+              CD4+ T Effector Memory         23875
                                CD4+ T Effector                 9353
                                CD4+ Reside

In [34]:
savepath = '/domino/edv/id-td-virology/Public_dataset/2022_Gut/for_Nadia/Final_Annotation_20220927.tsv'
df_2.to_csv(savepath, sep = '\t')